# Experimental QKD Protocol Implementation

## 1. Alice - The Sender Part
In this case, Ashesh (Alice) is responsible for preparing the qubits (polarized photons) and sending them to Bob (Ayaz).

### Laser Source
- A continuous laser source generates light, which is polarized using a polarizer.

### Polarizer and Intensity Modulation
- **Polarizer Control**: Alice uses a rotating polarizer, controlled by an Arduino and servo motor, to set the polarization state of the photons (qubits).
- **Polarization Basis**: The rotation angle of the polarizer determines the polarization basis (Rectilinear or Diagonal), corresponding to the binary bits (0 or 1).
- **Intensity Modulation**: The intensity of the laser is modulated (using a filter or adjustable power source) to encode information:
  - A low intensity represents "0".
  - A high intensity represents "1".

### Action Steps
1. The Arduino generates random polarization angles (0°, 90°, 45°, 135°), controlling the servo motor to rotate the polarizer accordingly.
2. The laser intensity is modulated to represent the bit value (0 or 1).

---

## 2. Bob - The Receiver Part
Ayaz (Bob) detects the incoming polarized light and decodes the qubits.

### Photodiode and Intensity Measurement
- A photodiode at Bob's side detects the light intensity and produces a small current proportional to the incident light.
- Since the current is small, a circuit using a 1 μF capacitor is employed to measure the charging time of the capacitor, which is proportional to the light intensity.

### Polarization Measurement
- Bob uses a polarizer on his side, controlled by another servo motor and Arduino, to select the measurement basis (Rectilinear or Diagonal).
- He rotates the polarizer to match his randomly chosen basis for each bit.
- The Arduino measures the charging time of the capacitor to determine the intensity and subsequently the bit value (0 or 1).

### Action Steps
1. **Random Basis Selection**: The Arduino at Bob’s end randomly selects a basis (Rectilinear or Diagonal) to measure the polarization.
2. **Intensity Measurement**: The Arduino calculates the capacitor charging time to measure the intensity of the light and decodes the bit.
3. **Store Measurement**: Bob stores the result of his measurement for each qubit.

---

## 3. Eavesdropper 
Ayush (Eve) is the eavesdropper who might attempt to intercept and measure the qubits.

- Eve can either measure the light with the same polarization basis as Alice or randomly choose a basis, leading to detectable errors.
- When Eve randomly changes the basis, the resulting intensity differences will be noticeable to Alice and Bob through increased errors in their keys.

---

## 4. Classical Channel - Basis Reconciliation
After transmission, Alice and Bob use a classical communication channel (e.g., Wi-Fi or a simple wired connection) to compare their chosen bases.

### Basis Announcement
- Alice and Bob announce which basis they used for each qubit (but not the actual bit values).

### Key Reconciliation
- They discard any measurements where their bases do not match. The remaining measurements form the shared key.


## Python Integration with Arduino
To automate this system, the Arduino microcontroller will handle the hardware control (polarizer rotation, intensity measurement) and communicate with a Python script running on a computer to process the data and complete the key exchange.

In [ ]:
import serial
import random
import time

# Establish communication with Arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)

def send_instruction_to_arduino(command):
    """Send command to Arduino and wait for a response"""
    arduino.write(bytes(command, 'utf-8'))
    time.sleep(0.1)
    response = arduino.readline().decode('utf-8').strip()
    return response

def generate_random_bases(length):
    """Generate random bases (0 for Rectilinear, 1 for Diagonal)"""
    return [random.choice([0, 1]) for _ in range(length)]

def generate_random_bits(length):
    """Generate random bit sequence (0s and 1s)"""
    return [random.choice([0, 1]) for _ in range(length)]

# Generate polarization bases and bit values
n = 100
alice_bases = generate_random_bases(n)
alice_bits = generate_random_bits(n)

# Alice sends polarization states and intensity to Arduino (acting as the polarizer)
for i in range(n):
    base = alice_bases[i]
    bit = alice_bits[i]
    # Send polarization base and intensity bit to Arduino
    # 0 = Rectilinear, 1 = Diagonal
    if base == 0:  # Rectilinear (0° or 90°)
        if bit == 0:
            command = "POLARIZE_0_LOW"  # 0° polarization, low intensity (logical 0)
        else:
            command = "POLARIZE_90_HIGH"  # 90° polarization, high intensity (logical 1)
    else:  # Diagonal (45° or 135°)
        if bit == 0:
            command = "POLARIZE_45_LOW"  # 45° polarization, low intensity (logical 0)
        else:
            command = "POLARIZE_135_HIGH"  # 135° polarization, high intensity (logical 1)
    
    # Send command to Arduino and wait for the response (confirmation)
    response = send_instruction_to_arduino(command)
    print(f"Alice: Sent {command}, Arduino Response: {response}")

# Bob measures the incoming qubits (using Arduino)
bob_bases = generate_random_bases(n)  # Bob randomly chooses bases to measure
bob_measurements = []

for i in range(n):
    base = bob_bases[i]
    
    # Send Bob's measurement base to Arduino
    if base == 0:
        measure_command = "MEASURE_RECTILINEAR"
    else:
        measure_command = "MEASURE_DIAGONAL"
    
    # Bob's Arduino measures the incoming qubit
    measurement = send_instruction_to_arduino(measure_command)
    bob_measurements.append(int(measurement))  # Store Bob's measurement result
    print(f"Bob: Measured using {'Rectilinear' if base == 0 else 'Diagonal'} basis, Result: {measurement}")

# Basis reconciliation: Alice and Bob announce their bases over a classical channel
# Compare bases and retain only the bits where the bases match
alice_key = []
bob_key = []

for i in range(n):
    if alice_bases[i] == bob_bases[i]:  # If bases match, keep the bit
        alice_key.append(alice_bits[i])
        bob_key.append(bob_measurements[i])

# Key reconciliation results
print(f"Alice's final key: {''.join(map(str, alice_key))}")
print(f"Bob's final key: {''.join(map(str, bob_key))}")

# Check for discrepancies (simulating eavesdropping detection)
def detect_discrepancies(alice_key, bob_key, threshold=0.1):
    """Check for errors between Alice's and Bob's keys"""
    differences = sum(a != b for a, b in zip(alice_key, bob_key))
    error_rate = differences / len(alice_key)
    return error_rate

error_rate = detect_discrepancies(alice_key, bob_key)
if error_rate > 0.1:
    print(f"Eavesdropping detected! Error rate: {error_rate * 100:.2f}%")
else:
    print(f"No eavesdropping detected. Error rate: {error_rate * 100:.2f}%")

# Proceed with the shared key if no eavesdropping is detected
if error_rate <= 0.1:
    # Example: Encrypt a message with the shared key using a one-time pad
    message = "HELLO"
    def one_time_pad_encrypt(message, key):
        binary_message = ''.join(format(ord(c), '08b') for c in message)
        encrypted_message = ''.join(str(int(binary_message[i]) ^ key[i % len(key)]) for i in range(len(binary_message)))
        return encrypted_message

    def one_time_pad_decrypt(encrypted_message, key):
        decrypted_bits = ''.join(str(int(encrypted_message[i]) ^ key[i % len(key)]) for i in range(len(encrypted_message)))
        decrypted_message = ''.join(chr(int(decrypted_bits[i:i + 8], 2)) for i in range(0, len(decrypted_bits), 8))
        return decrypted_message

    # Encrypting message using the final key
    print(f"Original Message: {message}")
    encrypted_message = one_time_pad_encrypt(message, alice_key)
    print(f"Encrypted Message: {encrypted_message}")

    # Decrypting message
    decrypted_message = one_time_pad_decrypt(encrypted_message, bob_key)
    print(f"Decrypted Message: {decrypted_message}")


# Detailed Steps:

Sending Polarization and Intensity Information:

Alice controls a laser source and polarizer through Arduino commands. The command is based on whether Alice uses a Rectilinear or Diagonal basis, and the intensity modulates to represent binary "0" or "1".
The Python script generates these commands and sends them to the Arduino (send_instruction_to_arduino), simulating the polarization and intensity settings for each qubit.
Bob’s Measurement:

Bob randomly selects a basis (Rectilinear or Diagonal) to measure each qubit.
The Arduino controls the polarizer at Bob’s end, and the photodiode measures the light intensity. The measurement is processed by calculating the charging time of the capacitor, which corresponds to the light intensity, and the result (0 or 1) is sent back to the Python script.
Basis Reconciliation:

Alice and Bob use a classical communication channel (in this case, simulated within the Python script) to reconcile their bases.
Only the qubits where Alice's and Bob's bases match are kept, and the rest are discarded. This forms their shared secret key.
Eavesdropping Detection:

A portion of the key is compared to detect any discrepancies, simulating the detection of an eavesdropper (Eve). If the error rate is too high, the communication is aborted.
Encryption and Decryption:

Once the key is established, the One-Time Pad encryption method is used to securely encrypt and decrypt a message between Alice and Bob.
Arduino Integration:
The Arduino should have a corresponding sketch to receive these commands (POLARIZE_0_LOW, MEASURE_RECTILINEAR, etc.) and perform actions like rotating the polarizer, measuring the photodiode, and sending results back to the Python script via serial communication.

You can expand the Arduino code to handle laser modulation, polarizer control (using a servo), and photodiode measurements based on the circuit design provided in the document.